In [2]:
import pandas as pd
from os import path
import pandas_profiling
pd.options.display.max_columns = 999
pd.options.display.max_rows = 1000
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import reverse_geocoder as rg
import numpy as np
from catboost import CatBoostClassifier
#from xgboost import XGBClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import seaborn as sns
import category_encoders as ce
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.externals.six import StringIO  
import pydot
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

/usr/local/lib/python3.5/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [72]:
def read_data():
    if path.exists('./train_withextra.csv') and path.exists('./test_withextra.csv'):
        print('here')
        train_df = pd.read_csv('./train_withextra.csv',header=0)
        test_df = pd.read_csv('./test_withextra.csv',header=0)
    else:
        train_df = pd.read_csv('./Train.csv',header=0)
        test_df = pd.read_csv('./Test.csv',header=0)
        train_df = modiefied_train_name(train_df)
        train_df =data_preprocessing(train_df)
        test_df =data_preprocessing(test_df)
        train_df.to_csv('./train_withextra.csv')
        test_df.to_csv('./test_withextra.csv')
    #train_report = pandas_profiling.ProfileReport(train_df)
    #train_report.to_file(output_file = './trainreport.html')
    #test_report = pandas_profiling.ProfileReport(test_df)
    #test_report.to_file(output_file = './testreport.html')
    return train_df,test_df

In [73]:
def one_hot_enco(train,test):
    cat_columns = ['train_name','source_address','destination_address','source_num','destination_num',\
                  'current_day', 'is_weekend', 'is_weekend_train']
    traindf_processed = pd.get_dummies(train,columns = cat_columns, \
                                          prefix_sep="__")
    
    cat_dummies = [col for col in traindf_processed if "__" in col and col.split("__")[0] in cat_columns]
    processed_columns = list(traindf_processed.columns[:])
    testdf_processed = pd.get_dummies(test, prefix_sep="__", 
                                   columns=cat_columns)
    
    # Remove additional columns
    for col in testdf_processed.columns:
        if ("__" in col) and (col.split("__")[0] in cat_columns) and col not in cat_dummies:
            print("Removing additional feature {}".format(col))
            testdf_processed.drop(col, axis=1, inplace=True)
    
    for col in cat_dummies:
        if col not in testdf_processed.columns:
            print("Adding missing feature {}".format(col))
            testdf_processed[col] = 0
            
    return traindf_processed,testdf_processed

In [79]:
def xgb_classifier(train_X,train_y,test_X,test_y):
    from sklearn.model_selection import RandomizedSearchCV
    params = {
            'min_child_weight': [1,2,3, 5, 6,7,8,9,10],
            'gamma': [0.1,0.3, 0.4,0.5,0.6,1, 1.5, 2,4, 5],
            'subsample': [0.1,0.2,0.4,0.6, 0.7,0.8, 1.0],
            'colsample_bytree': [0.1,0.3,0.6, 0.8, 1.0],
            'max_depth': [3, 4, 5,7,9]
            }
    xgb = XGBClassifier(learning_rate=0.01, n_estimators=1000, objective='multi:softmax',
                        silent=True, nthread=1,num_class = 3,num_boost_round=999,early_stopping_rounds=50)
    folds = 10
    param_comb = 15

    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

    random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, n_jobs=4, cv=skf.split(train_X,train_y), verbose=3, random_state=1001 )

    random_search.fit(train_X, train_y)
    print(random_search.cv_results_)
    print('\n Best estimator:')
    print(random_search.best_estimator_)
    #print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
    #print(random_search.best_score_ * 2 - 1)
    print('\n Best hyperparameters:')
    print(random_search.best_params_)
    print(classification_report(train_y,random_search.predict(train_X)))
    print(classification_report(test_y,random_search.predict(test_X)))
    return random_search

In [80]:
def convert24(str1): 
      
    # Checking if last two elements of time 
    # is AM and first two elements are 12 
    if str1[-2:] == "AM" and str1[:2] == "12":
        #print(str1[2:-2])
        return float('0' + '.'+ str1[3:5]) 
          
    # remove the AM     
    elif str1[-2:] == "AM": 
        return float(str1[:2] + '.' + str1[3:5]) 
      
    # Checking if last two elements of time 
    # is PM and first two elements are 12    
    elif str1[-2:] == "PM" and str1[:2] == "12": 
        return float(str1[:2] + '.' + str1[3:5])  
          
    else: 
          
        # add 12 to hours and remove PM 
        temp = str(int(str1[:2]) + 12) + str1[2:8]
        return float(temp[:2] + '.' + temp[3:5])
        #return str(temp[:2] + '.'+ temp[3:5])

def modiefied_train_name(df):
    df.loc[df['train_name'] == 'UNDEFINED', 'train_name'] = 'ICZVZS'
    return df

def convertdecimal(str1):
    return float(str1[:2] + '.' + str1[3:4])

def columns_tobe_ignored(df,cols):
    df.drop(columns =cols, inplace = True)
    return df

def label_encoding(train,test):
    #  label encoding to all the columns
    le=LabelEncoder()
    for col in test.columns.values:
        if (test[col].dtypes=='object') or (test[col].dtypes == 'bool'):
            data=train[col].append(test[col])
            le.fit(data.values)
            train[col]=le.transform(train[col])
            test[col]=le.transform(test[col])            
    return train,test

def label_encoding_target(train):
    le=LabelEncoder()
    train['target']=le.fit_transform(train['target'])
    return le,train

def lat_long_2_name(lat,long):
    if not (np.isnan(lat) or np.isnan(long)):
        coordinates = (lat,long)
        #print(coordinates)
        rt = rg.search(coordinates)
        #print(rt)
        return rt[0].get('name')
    else:
        return ''
    
def trains_on_weekends(df):
    first = df.loc[df['is_weekend'] == True]
    weekend_trains = first.train_name.unique()
    second = df.loc[df['is_weekend'] == False]
    weekday_trains = second.train_name.unique()
    li = [x for x in weekend_trains if x not in weekday_trains]
    inex = df['train_name'].isin(li)
    df['is_weekend_train'] = inex
    return df

def non_intersection_target(df):
    low = df.loc[df['target'] == 'low']
    low_trains = low.train_name.unique()
    medium = df.loc[df['target'] == 'medium']
    medium_trains = medium.train_name.unique()
    high = df.loc[df['target'] == 'high']
    high_trains = high.train_name.unique()
    print('intersection of all 3 trains')
    #print(set(low_trains) & set(medium_trains) & set(high_trains))
    print('\n')
    print('exclusive low')
    #print(set(low_trains) - (set(medium_trains) | set(high_trains)))
    print('\n')
    print('exclusive high')
    #print(set(high_trains) - (set(low_trains) | set(medium_trains)))
    print('\n')
    print('exclusive medium')
    
        
#dataset3.head() 
def exploratory_analyis(df):
    sns.FacetGrid(df,hue = 'target', size = 5).map(sns.distplot,'source_num').add_legend()

def hash_encoding(train,test):
    categorical_columns = ['current_day', 'is_weekend', 'is_weekend_train']
    #for col in categorical_columns:
        #if (test[col].dtypes=='object') or (test[col].dtypes == 'bool'):
        #    categorical_columns.append(col)
    he = ce.HashingEncoder(cols=categorical_columns, n_components=len(categorical_columns))
    train_he = he.fit_transform(train)
    test_he = he.transform(test)
    return he,train_he,test_he

In [81]:
def data_preprocessing(df):
    df['source_num'] = df.source_name.str.partition("station$",True)[2]
    df['destination_num'] = df.destination_name.str.partition("station$",True)[2]
    df['current_decimal_time'] = df.apply(lambda x: convert24(x['current_time']), axis=1)
    df['source_address'] = df.apply(lambda x: lat_long_2_name(x['latitude_source'],x['longitude_source']), axis=1)
    df['destination_address'] = df.apply(lambda x: lat_long_2_name(x['latitude_destination'],x['longitude_destination']), axis=1)
    return df

In [82]:
def datascaling(train_frame,test_frame):
    from sklearn.preprocessing import StandardScaler,MinMaxScaler
    #longitude_source,latitude_source,mean_halt_times_source,longitude_destination,latitude_destination,mean_halt_times_destination
#current_week,source_num,destination_num,current_decimal_time,
    columns = ['current_week', 'latitude_destination', 'latitude_source', 'longitude_destination', 'longitude_source',
       'mean_halt_times_destination', 'mean_halt_times_source','current_decimal_time']
    std_scale = MinMaxScaler().fit(train_frame[columns])
    train_frame[columns] = std_scale.transform(train_frame[columns])
    test_frame[columns]  = std_scale.transform(test_frame[columns])
    return train_frame,test_frame

In [83]:
train_frame,test_frame = read_data()
train_original = train_frame.copy()
test_original = test_frame.copy()
train_frame = trains_on_weekends(train_frame)
test_frame = trains_on_weekends(test_frame)
y = train_frame['target']
train_frame = train_frame.drop(['target'],axis=1)
#he,train_he_frame,test_he_frame = hash_encoding(train_frame,test_frame)
#print(train_he_frame.tail(50))
print(train_he_frame.columns)
train_he_frame,test_he_frame = one_hot_enco(train_frame,test_frame)
#print(train_he_frame.columns)
#print(test_he_frame.columns)

train_frame = columns_tobe_ignored(train_he_frame,['id_code','current_date','current_time','source_name','destination_name',\
                                                'country_code_source','current_year','country_code_destination'])

test_frame = columns_tobe_ignored(test_he_frame,['id_code','current_date','current_time','source_name','destination_name',\
                                                'country_code_source','current_year','country_code_destination'])



#print('Fore test\n')
#test_frame = data_preprocessing(test_frame)
#print(test_frame.columns)

#print(train_frame.info())
#longitude_source,latitude_source,mean_halt_times_source,longitude_destination,latitude_destination,mean_halt_times_destination
#current_week,source_num,destination_num,current_decimal_time,

here
Index(['col_0', 'col_1', 'col_2', 'longitude_source', 'latitude_source',
       'mean_halt_times_source', 'longitude_destination',
       'latitude_destination', 'mean_halt_times_destination', 'current_week',
       ...
       'destination_address__Waregem', 'destination_address__Waremme',
       'destination_address__Wemmel', 'destination_address__Wetteren',
       'destination_address__Wichelen', 'destination_address__Zaventem',
       'destination_address__Zeebrugge', 'destination_address__Zemst',
       'destination_address__Zingem', 'destination_address__Zottegem'],
      dtype='object', length=868)
Removing additional feature train_name__ICEZZ
Removing additional feature train_name__ICRAU
Removing additional feature train_name__ICRWZ
Removing additional feature train_name__ICRXS
Removing additional feature train_name__ICRXU
Removing additional feature train_name__ICRYR
Removing additional feature train_name__ICRYUA
Removing additional feature train_name__ICRYUX
Removing addi

Removing additional feature train_name__PSUAW
Removing additional feature train_name__PSWAZ
Removing additional feature train_name__PSXAU
Removing additional feature train_name__PSXAW
Removing additional feature train_name__PSXAX
Removing additional feature train_name__PSXAY
Removing additional feature train_name__PSXUW
Removing additional feature train_name__PSXUX
Removing additional feature train_name__PTAAU
Removing additional feature train_name__PTARZ
Removing additional feature train_name__PTAVX
Removing additional feature train_name__PTRAZ
Removing additional feature train_name__PTRRT
Removing additional feature train_name__PTSAZ
Removing additional feature train_name__PTTYY
Removing additional feature train_name__PTTYZ
Removing additional feature train_name__PTUAX
Removing additional feature train_name__PTVSV
Removing additional feature train_name__PTVZW
Removing additional feature train_name__PTVZX
Removing additional feature train_name__PTWSR
Removing additional feature train_

Removing additional feature source_num__618
Removing additional feature source_num__621
Removing additional feature source_num__637
Removing additional feature source_num__639
Removing additional feature destination_num__15
Removing additional feature destination_num__32
Removing additional feature destination_num__33
Removing additional feature destination_num__49
Removing additional feature destination_num__94
Removing additional feature destination_num__98
Removing additional feature destination_num__101
Removing additional feature destination_num__102
Removing additional feature destination_num__103
Removing additional feature destination_num__104
Removing additional feature destination_num__135
Removing additional feature destination_num__152
Removing additional feature destination_num__154
Removing additional feature destination_num__161
Removing additional feature destination_num__169
Removing additional feature destination_num__172
Removing additional feature destination_num__1

Adding missing feature train_name__SYXTVT
Adding missing feature train_name__SYXTVU
Adding missing feature train_name__SYXURY
Adding missing feature train_name__SYXUSR
Adding missing feature train_name__SYXUSX
Adding missing feature train_name__SZAXSSV
Adding missing feature train_name__SZAXSTR
Adding missing feature train_name__SZAXSUU
Adding missing feature train_name__SZAXSVT
Adding missing feature train_name__SZAYASA
Adding missing feature train_name__SZAYASR
Adding missing feature train_name__SZAYATS
Adding missing feature train_name__SZAYAVS
Adding missing feature train_name__SZAYAVV
Adding missing feature train_name__SZZRRU
Adding missing feature train_name__SZZRSA
Adding missing feature train_name__SZZRSW
Adding missing feature train_name__SZZRSX
Adding missing feature train_name__SZZRUA
Adding missing feature train_name__SZZRUT
Adding missing feature train_name__SZZRVV
Adding missing feature train_name__SZZTSR
Adding missing feature train_name__SZZTSS
Adding missing feature tr

Adding missing feature destination_num__219
Adding missing feature destination_num__220
Adding missing feature destination_num__222
Adding missing feature destination_num__223
Adding missing feature destination_num__233
Adding missing feature destination_num__234
Adding missing feature destination_num__243
Adding missing feature destination_num__245
Adding missing feature destination_num__246
Adding missing feature destination_num__249
Adding missing feature destination_num__252
Adding missing feature destination_num__256
Adding missing feature destination_num__262
Adding missing feature destination_num__268
Adding missing feature destination_num__271
Adding missing feature destination_num__273
Adding missing feature destination_num__292
Adding missing feature destination_num__345
Adding missing feature destination_num__359
Adding missing feature destination_num__381
Adding missing feature destination_num__383
Adding missing feature destination_num__385
Adding missing feature destinati

In [84]:
train_frame.fillna(train_frame.mean(), inplace=True)
#le,train_frame = label_encoding_target(train_frame)
#y = label_binarize(y, classes=[0, 1, 2])
train_frame,test_frame = datascaling(train_frame,test_frame)
#train_he_frame,test_he_frame = hash_encoding(train_frame,test_frame)
#y = train_frame['target']
#train_X = train_frame.drop(['target'],axis=1)
columns_to_scale  = ['gre', 'gpa']

#test_X = test_frame.drop(['Unnamed: 0'],axis=1)
#train_he_frame,test_he_frame = train_X,test_X
#hashenc,train_he_frame,test_he_frame = hash_encoding(train_X,test_X)
#print(hashenc.get_feature_names())
#train_frame = train_frame[train_frame.longitude_source > 0]

In [86]:
#print(train_he_frame.head(30))
#print(y)
X_train, X_vali, y_train, y_vali = train_test_split(train_frame, y, test_size=0.20)
#modelde = decision_tree_classifier(X_train, y_train,X_vali, y_vali)
#pre = model.predict(test_he_frame)
#print('\n')
#modelra = ranom_forest_classifier(X_train, y_train,X_vali, y_vali)
#modelknn = knn_classifier(X_train, y_train,X_vali, y_vali)
modelxgb =XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1.0,
              early_stopping_rounds=100, gamma=0.4, learning_rate=0.01,
              max_delta_step=0, max_depth=9, min_child_weight=3, missing=None,
              n_estimators=1000, n_jobs=1, nthread=1, num_boost_round=999,
              num_class=3, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=0.8, verbosity=1)


#,X_vali, y_vali)
#modelde = DecisionTreeClassifier(random_state=4,min_samples_split = 13, max_features = 15, min_samples_leaf = 5)
modelxgb.fit(X_train, y_train,verbose=3)

#dot_data = StringIO() 
#tree.export_graphviz(modelde, out_file=dot_data)
#tree.export_graphviz(modelde, out_file=dot_data, feature_names=hashenc.get_feature_names())
#graph = pydot.graph_from_dot_data(dot_data.getvalue()) 

#graph[0].write_pdf("./hitachi.pdf")  # must access graph's first element
#modelxgb = xgb_classifier(X_train, y_train,X_vali, y_vali)
#modelgbc = gbc_classifier(X_train, y_train,X_vali, y_vali)
#submission = pd.DataFrame({'id_code':test_original['id_code'],'target':modelde.predict(test_he_frame)})
#submission.to_csv('./submission.csv')


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1.0,
              early_stopping_rounds=100, gamma=0.4, learning_rate=0.01,
              max_delta_step=0, max_depth=9, min_child_weight=3, missing=None,
              n_estimators=1000, n_jobs=1, nthread=1, num_boost_round=999,
              num_class=3, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=0.8, verbosity=1)

In [87]:
print(classification_report(y_train,modelxgb.predict(X_train)))
print(classification_report(y_vali,modelxgb.predict(X_vali)))

              precision    recall  f1-score   support

        high       0.95      0.97      0.96       313
         low       0.98      0.97      0.98       426
      medium       0.98      0.95      0.96       288

    accuracy                           0.97      1027
   macro avg       0.97      0.97      0.97      1027
weighted avg       0.97      0.97      0.97      1027

              precision    recall  f1-score   support

        high       0.52      0.57      0.55        79
         low       0.64      0.72      0.68       106
      medium       0.42      0.31      0.35        72

    accuracy                           0.56       257
   macro avg       0.53      0.53      0.53       257
weighted avg       0.54      0.56      0.55       257



In [70]:
submission = pd.DataFrame({'id_code':test_original['id_code'],'target':modelxgb.predict(test_frame)})
submission.to_csv('./submission.csv',index = False)